# ***Matrix Profile*** (STUMP)

In this notebook, the Matrix Profile is evaluated but after being tuned. 
For each time series, before evaluation, the algorithm's parameters are tuned using a (presumably) similar time series. For each time series selected/constructed for evaluation there was a similar one selected/constructed meant for tuning.

The optimization criterion for the parameter selection is the AUC score.

The implementation is the STUMP function of the stumpy module.

## Imports

In [50]:
import math
import os
import sys
import json
from time import time
import time as timemod
from collections import defaultdict

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

In [4]:
from TSB_UAD.models.distance import Fourier
from TSB_UAD.models.feature import Window
from TSB_UAD.utils.slidingWindows import find_length, plotFig, printResult

from TSB_UAD.models.iforest import IForest

## Data Pre-Processing

In [8]:
# Load the data for the evaluation.
all_data = []

with open('dataset.pkl', 'rb') as f:
    data = pkl.load(f)

all_data.extend(data['evaluation']['single_normality'])
all_data.extend(data['evaluation']['double_normality'])
all_data.extend(data['evaluation']['triple_normality'])
all_data.extend(data['evaluation']['quadruple_normality'])
name_to_eval_series = {ts['Name']:ts for ts in all_data}

tuning_data = []
tuning_data.extend(data['tuning']['single_normality'])
tuning_data.extend(data['tuning']['double_normality'])
tuning_data.extend(data['tuning']['triple_normality'])
tuning_data.extend(data['tuning']['quadruple_normality'])
name_to_tune_series = {ts['Name']:ts for ts in tuning_data}

In [1]:
def preprocess_ts(timeseries, windows_per_batch=150):  
    # === Pre-processing steps ===

    # Prepare data for unsupervised method
    name = timeseries['Name']

    data = timeseries['data']
    max_length = data.shape[0]
    label = timeseries['labels']

    slidingWindow = find_length(data)
    X_data = Window(window=slidingWindow).convert(data).to_numpy()

    # Take the series and batch it.
    batched_data = []

    i = 0
    flag = True
    # Keep taking batches until the point at which no new windows can be taken.
    while i < len(data) and flag:
        # The data batches begin at the index indicated. If first batch, then the beginning of the time series.
        batch_samples_begin = i

        # The data batches end at the index where `windows_per_batch` can be *completely* extracted since the batch beginning. 
        # Formula: 
        #   i: current beginning of batch / offset
        #   + slidingWindow: to have enough samples extract one window
        #   + windows_per_batch: to have enough samples to extract the rest of the windows
        #   - 1: because the first window extracted is counted twice
        batch_samples_end = i + windows_per_batch + slidingWindow - 1
        
        # Guard against the ending of the time series where a full batch cannot be formed.
        if batch_samples_end > len(data):
            batch_samples_end = len(data)
            flag = False
 
        # Guard against case where the batch cannot hold even one window.
        if len(data[batch_samples_begin:batch_samples_end]) < slidingWindow:
            break

        batched_data.append(data[batch_samples_begin:batch_samples_end])

        # The next batch starts at the point where a new window be created after the last window of the last batch.
        # So, end of the previous window - length of window = start of the last window.
        #   start of the last window + 1 = start of the first window of the next batch.
        i = batch_samples_end - slidingWindow + 1

    # Take the windows and batch them.
    batched_X_data = []
    i = 0
    while i < len(X_data):
        begin = i
        end = i + windows_per_batch
        if end > len(X_data):
            end = len(X_data)

        batched_X_data.append(X_data[begin:end])
        i += windows_per_batch

    print(f'Time-Series name: {name}')
    print("Estimated Subsequence length: ", slidingWindow)
    print()
    
    # Store the pre-processed variables in the new dictionary
    return {
        'name': name,
        'data': data,
        'label': label,
        'slidingWindow': slidingWindow,
        'X_data': X_data,
        'batched_X_data': batched_X_data,
        'batched_data': batched_data,
        'Time series length': len(data),
        'Number of abnormal points': list(label).count(1)
    }

## 1. Offline Running
The STUMP algorithm is run offline, with access to all subsequences.

In [12]:
import stumpy
modelName = 'STUMP'

In [43]:
np.product([len(pl) for pl in param_grid.items()])

8

In [46]:
[len(pl) for pl in param_grid.values()]

[6, 5, 2]

In [53]:
results = []

# Parameters for tuning.
param_grid = {
    # Using the estimated window length from the autocorrelation, define alternate window sized as fractions/multiples of that.
    'window_length_modifier': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0], 
    # Number of trees in the isolation forest.
    'k': [1, 2, 5, 10, 20],
    'normalized': [True, False]
}

params_to_AUC = defaultdict(dict)

total = np.product([len(pl) for pl in param_grid.values()])

for timeseries in (p := tqdm(tuning_data)):
    ts = preprocess_ts(timeseries=timeseries)
    name = ts['name']

    p.set_description(name)
    x = ts['data']

    i = 0
    best_AUC = 0
    # Initial Best parameters are the defaults.
    best_params = (1, 1, True)
    for window_length_modifier in param_grid['window_length_modifier']:
        for k in param_grid['k']:
            for normalize in param_grid['normalized']:
                # Prevent too small windows.
                window_size = max(10, int(window_length_modifier * ts['slidingWindow']))
                score_ = stumpy.stump(T_A=x, m=window_size, k=k, ignore_trivial=True, normalize=normalize)
                score = score_.T[k-1]
                
                score = [s if s != np.inf else 0 for s in score]
                score = np.array(score)
                score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
                score = np.array([score[0]]*math.ceil((window_size-1)/2) + list(score) + [score[-1]]*((window_size-1)//2))
                
                AUC = printResult(ts['data'], ts['label'], score, window_size, ts['name'], modelName)[0]

                params_to_AUC[name][(window_length_modifier, k, normalize)] = AUC

                if AUC > best_AUC:
                    best_AUC = AUC
                    best_params = (window_length_modifier, k, normalize)

                i+=1
                print(f"\r[{i}/{total}]{name}  --  Best AUC = {best_AUC} for: {best_params}", end='')
    print()
    print(f"{name}  --  Best AUC = {best_AUC} for: {best_params}")

    # Evaluate evaluation time series with selected parameters.
    eval_series_name = ''.join([n if n!='2' else '1' for n in name]).replace('10k', '20k')  # Replace 2s with 1s.
    ts = preprocess_ts(timeseries=name_to_eval_series[eval_series_name])
    x = ts['data']

    window_size = max(10, int(ts['slidingWindow'] * best_params[0]))
    k = best_params[1]
    normalize = best_params[2]

    t0 = time()
    score_ = stumpy.stump(T_A=x, m=window_size, k=k, ignore_trivial=True, normalize=normalize)
    t1 = time()
    score = score_.T[k-1]
    
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((window_size-1)/2) + list(score) + [score[-1]]*((window_size-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, window_size, ts['name'], modelName)
    print(f"{eval_series_name}  --  Eval AUC = {L[0]}")
    results.append([name] + L + [t1-t0, len(x)])

    print()
    print('----------------------------------------------------------------')
    timemod.sleep(1)

ECG2_20k:   0%|          | 0/9 [00:00<?, ?it/s]

Time-Series name: ECG2_20k
Estimated Subsequence length:  89

[60/60]ECG2_20k  --  Best AUC = 0.9279824595528058 for: (2.0, 20, False)
ECG2_20k  --  Best AUC = 0.9279824595528058 for: (2.0, 20, False)
Time-Series name: ECG1_20k
Estimated Subsequence length:  100

ECG1_20k  --  Eval AUC = 0.9678807531982176

----------------------------------------------------------------


IOPS2:  11%|█         | 1/9 [01:20<10:39, 79.97s/it]   

Time-Series name: IOPS2
Estimated Subsequence length:  288

[60/60]IOPS2  --  Best AUC = 0.872010259212044 for: (0.1, 10, False)
IOPS2  --  Best AUC = 0.872010259212044 for: (0.1, 10, False)
Time-Series name: IOPS1
Estimated Subsequence length:  288

IOPS1  --  Eval AUC = 0.7787645976272559

----------------------------------------------------------------


SMD2:  22%|██▏       | 2/9 [01:49<05:53, 50.50s/it] 

...

Time-Series name: ECG2+IOPS2
Estimated Subsequence length:  89

[60/60]ECG2+IOPS2  --  Best AUC = 0.8940744322978126 for: (2.0, 20, False)
ECG2+IOPS2  --  Best AUC = 0.8940744322978126 for: (2.0, 20, False)
Time-Series name: ECG1+IOPS1
Estimated Subsequence length:  100

ECG1+IOPS1  --  Eval AUC = 0.7964428659801388

----------------------------------------------------------------


SMD2+Occupancy2:  56%|█████▌    | 5/9 [07:13<06:23, 95.87s/it]

Time-Series name: SMD2+Occupancy2
Estimated Subsequence length:  125

[60/60]SMD2+Occupancy2  --  Best AUC = 0.9153108878016463 for: (5.0, 1, False)
SMD2+Occupancy2  --  Best AUC = 0.9153108878016463 for: (5.0, 1, False)
Time-Series name: SMD1+Occupancy1
Estimated Subsequence length:  125

SMD1+Occupancy1  --  Eval AUC = 0.9241386413698732

----------------------------------------------------------------


ECG2+IOPS2+Occupancy2:  67%|██████▋   | 6/9 [10:14<06:14, 124.67s/it]

Time-Series name: ECG2+IOPS2+Occupancy2
Estimated Subsequence length:  89

[17/60]ECG2+IOPS2+Occupancy2  --  Best AUC = 0.8547573336428513 for: (0.5, 5, False))

In [10]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [11]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.635414,0.084577,21105.0,63.399095,229900
1,ECG1_20k,0.857905,0.133333,675.0,0.288338,20000
2,IOPS1,0.719323,0.189320,206.0,0.279814,8784
3,SMD1,0.472878,0.000000,2694.0,0.917965,28479
4,Occupancy1,0.169576,0.000000,972.0,0.072367,2665
5,ECG1+IOPS1,0.759207,0.098751,881.0,0.962583,28784
6,SMD1+Occupancy1,0.377355,0.000000,3666.0,1.031239,31144
7,ECG1+IOPS1+Occupancy1,0.765244,0.058284,1853.0,1.064308,31449
8,SMD1+ECG1+Occupancy1,0.576887,0.000000,4341.0,1.995396,51144
9,ECG1+IOPS1+SMD1+Occupancy1,0.613329,0.000000,4547.0,2.595874,59928


In [12]:
df.to_csv('Results/Static/STUMP-results.csv', index=False)

## 2. Online Running: Variant 1
The STUMP algorithm is run online, with access to limited subsequences. In this variant, the algorithm only has access to the subsequences of a given (current) batch.

In [13]:
results = []

for name in (p := tqdm(preprocessed_dict.keys())):
    p.set_description(name)
    ts = preprocessed_dict[name]
    window_size = ts['slidingWindow']
    
    k = 1
    score = []
    t0 = time()
    for batch in tqdm(ts['batched_data'], desc='Processing Batch'):
        score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)
        score.extend(score_.T[k-1])
    t1 = time()

    # In some combinations of batch size and window size, windows overlap with all closest-distance candidates and cannot be scored.
    # In this case, inf is returned. To fix this, any instances of infinite distances are replaced with zero distance.
    score = [s if s != np.inf else 0 for s in score]
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([name] + L + [t1-t0, len(x)])

ECG1+IOPS1+SMD1+Occupancy1: 100%|██████████| 10/10 [00:24<00:00,  2.49s/it]


In [14]:
columns = ['Name'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precision@k', 'RangeAUC', 'Time', 'Number of Windows']
df = pd.DataFrame(results, columns=columns)

In [15]:
df['Number of anomalies'] = df['Name'].apply(lambda x: np.sum(preprocessed_dict[x]['label']))
df[['Name', 'AUC', 'Precision@k', 'Number of anomalies', 'Time', 'Number of Windows']]

,Name,AUC,Precision@k,Number of anomalies,Time,Number of Windows
0,ECG1,0.696418,0.0,21105.0,11.109997,59928
1,ECG1_20k,0.689407,0.0,675.0,0.979199,59928
2,IOPS1,0.596923,0.0,206.0,0.416120,59928
3,SMD1,0.556622,0.0,2694.0,1.345566,59928
4,Occupancy1,0.186049,0.0,972.0,0.129305,59928
5,ECG1+IOPS1,0.708663,0.0,881.0,1.402862,59928
6,SMD1+Occupancy1,0.441063,0.0,3666.0,0.981996,59928
7,ECG1+IOPS1+Occupancy1,0.499768,0.0,1853.0,1.003361,59928
8,SMD1+ECG1+Occupancy1,0.542541,0.0,4341.0,1.617278,59928
9,ECG1+IOPS1+SMD1+Occupancy1,0.525118,0.0,4547.0,1.872202,59928


In [16]:
df.to_csv('Results/Variant_1/STUMP-results.csv', index=False)